In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/geoffrey-hinton-fellowship-hackathon-2/Training Data Target.csv
/kaggle/input/geoffrey-hinton-fellowship-hackathon-2/Test Data.csv
/kaggle/input/geoffrey-hinton-fellowship-hackathon-2/Training Data.csv
/kaggle/input/geoffrey-hinton-fellowship-hackathon-2/Sample Submission.csv


In [2]:
import seaborn as sns 
import matplotlib.pyplot  as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import category_encoders as ce

In [3]:
df_t=pd.read_csv("../input/geoffrey-hinton-fellowship-hackathon-2/Training Data.csv")
df_T=pd.read_csv("../input/geoffrey-hinton-fellowship-hackathon-2/Test Data.csv")

In [ ]:
#df_T=df_T.drop(columns='Unnamed: 0')


In [7]:
a=pd.unique(df_T['user_id'])
print(len(a))

19981


In [ ]:

sns.scatterplot(x='category',y='aov',data=df_T)

In [ ]:
#df_s=(df_T.groupby(by=['user_id','category'])['aov'].sum().reset_index())
#df_t=(df_t.groupby(by=['user_id','category'])['aov'].sum().reset_index())
#df_s

In [8]:
table1=pd.pivot_table(df_t,index=['user_id','category'],aggfunc='count')
table1

aov
user_id category                      
1       Fashion                      2
        Fitness                      2
        Groceries                    1
        Phones                       1
        Toys                         1
...                                ...
50075   Phones                       1
50078   Books                        1
50079   Kitchen cleaning Supplies    1
50080   Fashion                      1
50082   Phones                       1

[159869 rows x 1 columns]

In [ ]:
product=pd.unique(df_t.category)
user_id=pd.unique(df_t.user_id)
print(user_id,product)
len(product),len(user_id)

In [ ]:
product=pd.unique(df_T.category)
user_id=pd.unique(df_T.user_id)
print(user_id,product)
len(product),len(user_id)

In [ ]:
sns.pairplot(df_t,hue='category')

In [ ]:
sns.pairplot(df_T,hue='category')

In [9]:
combine_order_rate=df_T.dropna(axis = 0, subset = ['category'])

product_orderCount = (combine_order_rate.
     groupby(by = ['category'])['aov'].
     count().
     reset_index().
     rename(columns = {'aov': 'totalOrderCount'})
     [['category', 'totalOrderCount']]
    )
product_orderCount.head(10)

,category,totalOrderCount
0,Back to School,8405
1,Beauty Products,2055
2,Board Games,1981
3,Books,3968
4,Consumer Durables,7984
5,Ereaders,2163
6,Fashion,40082
7,Fitness,1961
8,Gaming,8109
9,Groceries,8559


In [10]:
product_with_totalOrderCount = combine_order_rate.merge(product_orderCount, left_on = 'category', right_on = 'category', how = 'left')
product_with_totalOrderCount.tail(10)

,Unnamed: 0,user_id,aov,category,totalOrderCount
172544,86132,4260,896.0,Books,3968
172545,86133,49488,330.0,Back to School,8405
172546,86134,39778,975.0,Fashion,40082
172547,86135,36793,2868.0,Groceries,8559
172548,86136,42131,647.0,Back to School,8405
172549,86137,12403,1841.0,Groceries,8559
172550,86138,31036,424.0,Back to School,8405
172551,86139,12652,617.0,Fashion,40082
172552,86140,35726,634.0,Kitchen cleaning Supplies,2042
172553,86141,19871,994.0,Fashion,40082


In [11]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)
print(product_orderCount['totalOrderCount'].describe())

count      18.0
mean     9586.3
std     12971.9
min      1255.0
25%      2000.8
50%      3065.5
75%      8520.5
max     44627.0
Name: totalOrderCount, dtype: float64


In [18]:
m=product_orderCount['totalOrderCount'].mean()
s=product_orderCount['totalOrderCount'].std()                                        
popularity_threshold1 = 1255 #25% of data
popularity_threshold2= m+3*s

rating_popular_product= product_with_totalOrderCount.query('totalOrderCount >= @popularity_threshold1 and totalOrderCount <= @popularity_threshold2 ')
rating_popular_product.head()

,Unnamed: 0,user_id,aov,category,totalOrderCount
0,0,43323,19115.0,Phones,44627
1,1,43323,29309.0,Phones,44627
2,2,43323,15293.0,Phones,44627
3,3,43323,23548.0,Phones,44627
4,4,43323,751.0,Fashion,40082


In [19]:
rating_popular_product.shape

(172554, 5)

In [20]:
product_features_df=rating_popular_product.pivot_table(index='category',columns='user_id',values='aov').fillna(0)
product_features_df.head(18)

user_id,2,3,8,10,13,15,17,18,20,22,...,49987,49989,49994,49995,49997,49998,50037,50050,50074,50089
category,,,,,,,,,,,,,,,,,,,,,
Back to School,0.0,0.0,763.0,0.0,711.0,0.0,694.0,0.0,574.5,482.0,...,0.0,522.0,0.0,332.0,0.0,0.0,0.0,315.0,0.0,949.0
Beauty Products,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Board Games,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Books,0.0,0.0,0.0,0.0,346.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,386.0,600.0,0.0,0.0,0.0,0.0,0.0
Consumer Durables,0.0,23785.0,29859.0,0.0,18990.0,0.0,31846.0,0.0,0.0,0.0,...,12000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ereaders,7696.0,8572.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11331.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fashion,582.5,561.0,733.5,681.0,597.0,666.7,719.4,934.3,820.0,896.5,...,622.3,0.0,0.0,645.7,803.0,572.5,0.0,0.0,0.0,0.0
Fitness,7035.0,0.0,2150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gaming,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1760.0,2594.0,0.0,...,0.0,0.0,0.0,391.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from scipy.sparse import csr_matrix

product_features_df_matrix = csr_matrix(product_features_df.values)
#print(product_features_df_matrix)
from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric='cosine',algorithm='brute',radius=0.4)
model_knn.fit(product_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', radius=0.4)

In [22]:
a=pd.DataFrame(product_features_df_matrix.todense())
a

,0,1,2,3,4,5,6,7,8,9,...,19971,19972,19973,19974,19975,19976,19977,19978,19979,19980
0,0.0,0.0,763.0,0.0,711.0,0.0,694.0,0.0,574.5,482.0,...,0.0,522.0,0.0,332.0,0.0,0.0,0.0,315.0,0.0,949.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,346.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,386.0,600.0,0.0,0.0,0.0,0.0,0.0
4,0.0,23785.0,29859.0,0.0,18990.0,0.0,31846.0,0.0,0.0,0.0,...,12000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,7696.0,8572.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11331.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,582.5,561.0,733.5,681.0,597.0,666.7,719.4,934.3,820.0,896.5,...,622.3,0.0,0.0,645.7,803.0,572.5,0.0,0.0,0.0,0.0
7,7035.0,0.0,2150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1760.0,2594.0,0.0,...,0.0,0.0,0.0,391.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,790.0,1694.0,0.0,0.0,0.0,0.0,...,1228.0,1643.0,0.0,0.0,0.0,0.0,1729.0,1352.0,0.0,0.0


In [ ]:
#product_features_df.head(18)

In [23]:
y=pd.DataFrame(columns=['category','predict3'])

for j in range(0,18):
    #query_index = np.random.choice(product_features_df.shape[0])
    query_index=j
    #print(query_index)
    distances, indices = model_knn.kneighbors(product_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 4)
    b=[]
    c=[]
    for i in range(0, len(distances.flatten())-1):
        if i == 0:
            print('\nRecommendations for {0}:\n'.format(product_features_df.index[query_index]))
            b+=[product_features_df.index[query_index]]
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, product_features_df.index[indices.flatten()[i]], distances.flatten()[i]))
            c+=[product_features_df.index[indices.flatten()[i]]]
    c=b+c
    d=b+[c]
    y=y.append(pd.Series(d,index=y.columns),ignore_index=True)   
y['pred3']=[','.join(map(str,i)) for i in y['predict3'] ]
y=y.drop(columns='predict3')


Recommendations for Back to School:

1: Phones, with distance of 0.49314884348179466:
2: Fashion, with distance of 0.5090684032513153:

Recommendations for Beauty Products:

1: Phones, with distance of 0.7403358025951705:
2: Fashion, with distance of 0.7490834280528387:

Recommendations for Board Games:

1: Phones, with distance of 0.7306681845878318:
2: Fashion, with distance of 0.7348287489729137:

Recommendations for Books:

1: Phones, with distance of 0.6302670020789793:
2: Fashion, with distance of 0.6345441923650454:

Recommendations for Consumer Durables:

1: Fashion, with distance of 0.43846887538818746:
2: Phones, with distance of 0.441314843820532:

Recommendations for Ereaders:

1: Phones, with distance of 0.7129106546566574:
2: Fashion, with distance of 0.7155352045486213:

Recommendations for Fashion:

1: Phones, with distance of 0.13477169026925184:
2: Home Decor, with distance of 0.3112883217594684:

Recommendations for Fitness:

1: Phones, with distance of 0.7425748615

In [ ]:
c=['1','2','3'] 
b=['3']
d=b+c
print(d)

In [24]:
y

,category,pred3
0,Back to School,"Back to School,Phones,Fashion"
1,Beauty Products,"Beauty Products,Phones,Fashion"
2,Board Games,"Board Games,Phones,Fashion"
3,Books,"Books,Phones,Fashion"
4,Consumer Durables,"Consumer Durables,Fashion,Phones"
5,Ereaders,"Ereaders,Phones,Fashion"
6,Fashion,"Fashion,Phones,Home Decor"
7,Fitness,"Fitness,Phones,Fashion"
8,Gaming,"Gaming,Phones,Fashion"
9,Groceries,"Groceries,Phones,Fashion"


In [25]:

prediction=df_T.merge(y,left_on = 'category', right_on = 'category', how = 'left')

prediction

,Unnamed: 0,user_id,aov,category,pred3
0,0,43323,19115.0,Phones,"Phones,Fashion,Home Decor"
1,1,43323,29309.0,Phones,"Phones,Fashion,Home Decor"
2,2,43323,15293.0,Phones,"Phones,Fashion,Home Decor"
3,3,43323,23548.0,Phones,"Phones,Fashion,Home Decor"
4,4,43323,751.0,Fashion,"Fashion,Phones,Home Decor"
...,...,...,...,...,...
172549,86137,12403,1841.0,Groceries,"Groceries,Phones,Fashion"
172550,86138,31036,424.0,Back to School,"Back to School,Phones,Fashion"
172551,86139,12652,617.0,Fashion,"Fashion,Phones,Home Decor"
172552,86140,35726,634.0,Kitchen cleaning Supplies,"Kitchen cleaning Supplies,Phones,Fashion"


In [ ]:
#prediction['pred3']=prediction['pred3'].fillna('Phones,Fashion,Home Decor')

In [29]:
duplicate_pred = prediction[prediction.duplicated([ 'user_id','category'])]
duplicate_pred = prediction[prediction.duplicated([ 'user_id'])]
duplicate_pred.shape

(87082, 4)

In [33]:
prediction=prediction.groupby(by=['user_id','pred3','category'])['aov'].sum().reset_index()
prediction=prediction.drop_duplicates(subset=[ 'user_id','pred3'])
prediction=prediction.sort_values(by=['user_id','aov'])
prediction=prediction.drop_duplicates(subset=[ 'user_id'] ,keep='last')
#prediction=(prediction.groupby(by=['user_id','category'])['aov'].sum().reset_index())

prediction=prediction.drop(columns=['aov','category'])
prediction

,user_id,pred3
0,2,"Phones,Fashion,Home Decor"
1,3,"Phones,Fashion,Home Decor"
2,8,"Phones,Fashion,Home Decor"
3,10,"Phones,Fashion,Home Decor"
4,13,"Phones,Fashion,Home Decor"
...,...,...
19976,49998,"Phones,Fashion,Home Decor"
19977,50037,"Groceries,Phones,Fashion"
19978,50050,"Groceries,Phones,Fashion"
19979,50074,"Kitchen cleaning Supplies,Phones,Fashion"


In [34]:
prediction.nunique()

user_id    19981
pred3         18
dtype: int64

In [ ]:
prediction.head(8)

In [35]:
product=pd.unique(prediction.pred3)
user_id=pd.unique(prediction.user_id)
print(user_id,product)
len(product),len(user_id)

[    2     3     8 ... 50050 50074 50089] ['Phones,Fashion,Home Decor' 'Laptops,Consumer Durables,Phones'
 'Consumer Durables,Fashion,Phones' 'Fashion,Phones,Home Decor'
 'Home Decor,Phones,Fashion' 'Toys,Phones,Fashion'
 'TVs,Consumer Durables,Fashion' 'Gaming,Phones,Fashion'
 'Groceries,Phones,Fashion' 'Ereaders,Phones,Fashion'
 'Painiting Supplies,Phones,Fashion' 'Fitness,Phones,Fashion'
 'Back to School,Phones,Fashion' 'Pet Supplies,Phones,Fashion'
 'Beauty Products,Phones,Fashion' 'Books,Phones,Fashion'
 'Board Games,Phones,Fashion' 'Kitchen cleaning Supplies,Phones,Fashion']


(18, 19981)

In [ ]:
def model_top3(test_data):
    combine_order_rate=test_data.dropna(axis = 0, subset = ['category'])
    product_orderCount = (combine_order_rate.
     groupby(by = ['category'])['aov'].
     count().
     reset_index().
     rename(columns = {'aov': 'totalOrderCount'})
     [['category', 'totalOrderCount']]
    )
    product_orderCount.head(10)
    product_with_totalOrderCount = combine_order_rate.merge(product_orderCount, left_on = 'category', right_on = 'category', how = 'left')
    product_with_totalOrderCount.tail(10)
    from scipy.sparse import csr_matrix
    product_features_df_matrix = csr_matrix(product_features_df.values)
    #print(product_features_df_matrix)
    from sklearn.neighbors import NearestNeighbors
    model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
    model_knn.fit(product_features_df_matrix)
    y=pd.DataFrame(columns=['category','predict3'])
    for j in range(0,18):
        #query_index = np.random.choice(product_features_df.shape[0])
        query_index=j
        #print(query_index)
        distances, indices = model_knn.kneighbors(product_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 4)
        b=[]
        c=[]
        for i in range(0, len(distances.flatten())-1):
            if i == 0:
                #print('\nRecommendations for {0}:\n'.format(product_features_df.index[query_index]))
                b+=[product_features_df.index[query_index]]
            else:
                #print('{0}: {1}, with distance of {2}:'.format(i, product_features_df.index[indices.flatten()[i]], distances.flatten()[i]))
                c+=[product_features_df.index[indices.flatten()[i]]]
        d=b+[c]
        y=y.append(pd.Series(d,index=y.columns),ignore_index=True) 
    y['pred3']=[','.join(map(str,i)) for i in y['predict3'] ]
    y=y.drop(columns='predict3')
    prediction=test_data.merge(y,left_on = 'category', right_on = 'category', how = 'left')
    df_s=(df_T.groupby(by=['user_id','category'])['aov'].sum().reset_index())
    prediction=prediction.drop(columns=['aov','category',])
    prediction=prediction.drop_duplicates(subset=[ 'user_id','pred3'])
    prediction=prediction.drop_duplicates(subset=[ 'user_id'])

    return prediction

In [ ]:
prediction=model_top3(df_T)  
prediction=prediction.reset_index(drop=True ,inplace=False)

In [36]:
prediction.to_csv('prediction5.csv',index=False)

In [ ]:
df_T.shape

In [37]:
prediction.head(14)

,user_id,pred3
0,2,"Phones,Fashion,Home Decor"
1,3,"Phones,Fashion,Home Decor"
2,8,"Phones,Fashion,Home Decor"
3,10,"Phones,Fashion,Home Decor"
4,13,"Phones,Fashion,Home Decor"
5,15,"Laptops,Consumer Durables,Phones"
6,17,"Phones,Fashion,Home Decor"
7,18,"Phones,Fashion,Home Decor"
8,20,"Phones,Fashion,Home Decor"
9,22,"Phones,Fashion,Home Decor"


### 